In [1]:
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
MODEL_PATH = './model/'
DATA_PATH = './data/'

2022-08-18 19:05:33.564861: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-18 19:05:33.565766: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [68]:
pur_data = np.load(DATA_PATH+'purchase100.npz')
features = pur_data['features']
labels = pur_data['labels']

197324

In [20]:
data = pd.read_csv('data/adult.data', na_values=["?"])
target_dataset = pd.DataFrame(data.iloc[1:10001,].values)
shadow_dataset = pd.DataFrame(data.iloc[15001:30001,].values)
attack_test_nonmembers = pd.DataFrame(data.iloc[11001:14001,].values)
attack_test_members = pd.DataFrame(data.iloc[2001:5001,].values)

In [2]:
target_dataset = pd.read_csv('data/adult_sds.csv', na_values=["?"])
shadow_dataset = pd.read_csv('data/adultODS10K_to_25K.csv', na_values=["?"])
attack_test_nonmembers = pd.read_csv('data/adultODS25K_to_28K.csv', na_values=["?"])
attack_test_members = pd.read_csv('data/adultODS1_to_3K.csv', na_values=["?"])

In [21]:
def read_data(data_name):
    with np.load(DATA_PATH + data_name) as f:
        train_x, train_y, test_x, test_y = [f['arr_%d' % i] for i in range(len(f.files))]
    return train_x, train_y, test_x, test_y

In [57]:
def transform_data(dataset):
    x = dataset.iloc[:,0:dataset.shape[1]-1] # seperate the feature column from class label
    y = dataset.iloc[:,-1] # label column

    dim=x.shape[1] # number of feature columns

    #num of classes
    num_classes=2

    for j in range(0,dim):
        if x.iloc[:,j].dtypes == object:   # transform categorical variables
            x.iloc[:,j] = x.iloc[:,j].astype('category') # change datatype to categorical
            x.iloc[:,j] = x.iloc[:,j].cat.codes # change from category name to category number
        else:  #transform numrical variables to standard scaler form 
            sc = StandardScaler()  
            val=np.array(x.iloc[:,j]).reshape(-1,1)
            std_data = sc.fit_transform(val)
            std_data = pd.DataFrame(std_data)
            x.iloc[:,j]=std_data

    y = y.astype('category') # change label to categorical
    y = y.cat.codes # change from category name to number

    x=np.array(x)
    y=np.array(y)
    
    
    y=to_categorical(y)
    
    return x, y, dim

In [23]:
def load_target_data(dataset, test_ratio):
    x, y, dim = transform_data(dataset)
    
    #trainX,testX, trainY, testY = train_test_split(x, y, test_size=test_ratio, random_state=0, stratify=y)
    trainX = x[0:7000,]
    testX = x[7000:,]
    trainY = y[0:7000,]
    testY = y[7000:,]
    return (trainX, trainY), (testX, testY), dim

In [24]:
def build_simple_mlp(n_class,pix,d):

    model = Sequential()
    model.add(Dense(256, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(256, kernel_regularizer=l2(0.01)))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    
    #model.add(Dense(248))
    #model.add(Activation("relu"))
    #model.add(Dropout(0.01))

    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=3
    
    return model, act_layer

In [35]:
def load_shadow_data(dataset, n_shadow, shadow_size, test_ratio):
    x, y, _ = transform_data(dataset)
    
    shadow_indices = np.arange(len(dataset))
    
   
    for i in range(n_shadow):
        shadow_i_indices = np.random.choice(shadow_indices, shadow_size, replace=False)
        shadow_i_x, shadow_i_y = x[shadow_i_indices], y[shadow_i_indices]
        trainX,testX, trainY, testY = train_test_split(shadow_i_x, shadow_i_y, test_size=test_ratio)
        print('shadow_i_trainX = ', len(trainX), 'shadow_i_trainY = ', len(trainY), 'shadow_i_testX = ', len(testX), 'shadow_i_testY = ', len(testY))
        
        np.savez(DATA_PATH + 'shadow_adult{}_data.npz'.format(i), trainX, trainY, testX, testY)

In [36]:
def train_shadow_models(n_shadow, n_class, dim, channel):
    full_sm_train_pred=[]
    full_sm_train_class=[]
    
    full_sm_test_pred=[]
    full_sm_test_class=[]
    
    full_clz_train=[]
    full_clz_test=[]
    
    members=[]
    nonmembers=[]

    for j in range(n_shadow):
        
        print("Shadow Model ", j)
        
        print('Training shadow model {}'.format(j))
        data = read_data('shadow_adult{}_data.npz'.format(j))
        x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test = data
        #print('x_shadow trian\n', x_shadow_train,'\n y_shadow trian\n', y_shadow_train, '\n x_shadow test\n', x_shadow_test, '\n y_shadow test\n', y_shadow_test)

        model, act_layer = build_simple_mlp(n_class,dim,channel)
            
            
        # fit model
        history = model.fit(x_shadow_train, y_shadow_train, epochs=EPS, batch_size=32, validation_data=(x_shadow_test, y_shadow_test), verbose=0)
    
        # evaluate model
        _, train_acc = model.evaluate(x_shadow_train, y_shadow_train, verbose=0)
        _, test_acc = model.evaluate(x_shadow_test, y_shadow_test, verbose=0)
        print("Shadow Train acc : ", (train_acc * 100.0),"Shadow Test acc : ", (test_acc * 100.0))

    
        #train SM
        sm_train_pred=model.predict(x_shadow_train, batch_size=32)
        sm_train_class=np.argmax(y_shadow_train,axis=1)
    
    
        #test SM
        sm_test_pred=model.predict(x_shadow_test, batch_size=32)
        sm_test_class=np.argmax(y_shadow_test,axis=1)
        
     
        full_sm_train_pred.append(sm_train_pred)        
        full_sm_train_class.append(sm_train_class)
        members.append(np.ones(len(sm_train_pred)))
        
        full_sm_test_pred.append(sm_test_pred)        
        full_sm_test_class.append(sm_test_class) 
        nonmembers.append(np.zeros(len(sm_test_pred)))


    full_sm_train_pred = np.vstack(full_sm_train_pred)
    full_sm_train_class = [item for sublist in full_sm_train_class for item in sublist]
    members = [item for sublist in members for item in sublist]
    
    full_sm_test_pred = np.vstack(full_sm_test_pred)
    full_sm_test_class = [item for sublist in full_sm_test_class for item in sublist]
    nonmembers = [item for sublist in nonmembers for item in sublist]
    
    shadow_train_performance=(full_sm_train_pred, np.array(full_sm_train_class))
    shadow_test_performance=(full_sm_test_pred, np.array(full_sm_test_class))


    ###atack data preparation
    attack_x = (full_sm_train_pred,full_sm_test_pred)
    #attack_x = np.vstack(attack_x)
    
    attack_y = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    #attack_y = np.concatenate(attack_y)
    #attack_y = attack_y.astype('int32')
    
    
    classes = (np.array(full_sm_train_class),np.array(full_sm_test_class))
    #classes = np.array([item for sublist in classes for item in sublist])


    attack_dataset = (attack_x,attack_y,classes)

            
    return  shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model

In [37]:
def define_attack_model(n_class):
    model = Sequential()
    
    model.add(Dense(1))
    model.add(Activation("relu"))
    
    model.add(Dense(1))
    model.add(Activation("relu"))

    model.add(Dense(n_class, activation='softmax'))

    # compile model
    opt = SGD(learning_rate=0.0001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [38]:
def load_attack_test_data(members, nonmembers):
    memberX, memberY, _ = transform_data(members)
    
    nonmemberX, nonmemberY, _ = transform_data(nonmembers)
    
    return memberX, memberY, nonmemberX, nonmemberY

In [39]:
def prety_print_result(mem, pred):
    tn, fp, fn, tp = confusion_matrix(mem, pred).ravel()
    print('TP: %d     FP: %d     FN: %d     TN: %d' % (tp, fp, fn, tn))
    if tp == fp == 0:
        print('PPV: 0\nAdvantage: 0')
    else:
        print('PPV: %.4f\nAdvantage: %.4f' % (tp / (tp + fp), tp / (tp + fn) - fp / (tn + fp)))

    return tp, fp, fn, tn, (tp / (tp + fp)), (tp / (tp + fn) - fp / (tn + fp)), ((tp+tn)/(tp+tn+fp+fn)),  (tp / (tp + fn))

In [65]:
def train_attack_model(attack_data, check_membership, n_hidden=50, learning_rate=0.01, batch_size=200, epochs=50, model='nn', l2_ratio=1e-7):

    x, y,  classes = attack_data

    train_x = x[0]
    train_y = y[0]
    test_x = x[1]
    test_y = y[1]
    train_classes = classes[0]
    test_classes = classes[1]
    
    
    checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership
    
    checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
    checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
    checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])
    
    train_indices = np.arange(len(train_x))
    test_indices = np.arange(len(test_x))
    unique_classes = np.unique(train_classes)


    predicted_membership, target_membership = [], []
    for c in unique_classes:
        print("Class : ", c)
        c_train_indices = train_indices[train_classes == c]
        c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
        c_test_indices = test_indices[test_classes == c]
        c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
        c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)        
        
        full_cx_data=(c_train_x,c_test_x)
        full_cx_data = np.vstack(full_cx_data)

        full_cy_data=(c_train_y,c_test_y)
        full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])
        full_cy_data = to_categorical(full_cy_data)

                
        
        classifier = define_attack_model(2)
        history = classifier.fit(full_cx_data, full_cy_data, epochs=EPS, batch_size=32, verbose=0)

        #get predictions on real train and test data
        c_indices = np.where(checkmem_class_status==c)
        c_pred_y = classifier.predict_classes(checkmem_prediction_vals[c_indices])
        c_target_y = checkmem_membership_status[c_indices]
        
       
        target_membership.append(c_target_y)
        predicted_membership.append(c_pred_y)

    target_membership=np.array([item for sublist in target_membership for item in sublist])
    predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])


    tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (target_membership,predicted_membership)   
    return tp, fp, fn, tn, precision, advj, acc, recall

In [31]:
# trian target model
per_class_sample=5000
channel=0   
EPS=200
act_layer=3
n_class = 2
VERBOSE = 0
test_ratio = 0.3

(target_trainX, target_trainY), (target_testX, target_testY), dim = load_target_data(target_dataset, test_ratio)
target_model,_ = build_simple_mlp (n_class,dim,channel)
#get_trained_keras_models(model, (target_trainX, target_trainY), (target_testX, target_testY), num_models=1)
history = target_model.fit(target_trainX, target_trainY, epochs=EPS, batch_size=32, verbose=VERBOSE)
score = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
print('\n', 'Model test accuracy:', score[1])


 Model test accuracy: 0.8136666417121887


In [60]:
# prepare attack test data

members = []
nonmembers = []

memberX, memberY, nonmemberX, nonmemberY = load_attack_test_data(attack_test_members, attack_test_nonmembers)

# member
target_model_member_pred = target_model.predict(memberX, batch_size=32)
target_model_member_class = np.argmax(memberY, axis=1)
target_model_member_pred = np.vstack(target_model_member_pred)
#target_model_member_class = [item for sublist in target_model_member_class for item in sublist]
members.append(np.ones(len(target_model_member_pred)))
members = [item for sublist in members for item in sublist]


# nonmember
target_model_nonmember_pred = target_model.predict(nonmemberX, batch_size=32)
target_model_nonmember_class = np.argmax(nonmemberY, axis=1)
target_model_nonmember_pred = np.vstack(target_model_nonmember_pred)
#target_model_nonmember_class = [item for sublist in target_model_nonmember_class for item in sublist]
nonmembers.append(np.zeros(len(target_model_nonmember_pred)))
nonmembers = [item for sublist in nonmembers for item in sublist]

full_attack_test_pred_val = (target_model_member_pred, target_model_nonmember_pred)
full_attack_test_mem_status = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
full_attack_test_class_status = (np.array(target_model_member_class),np.array(target_model_nonmember_class))

print('\n pred', full_attack_test_pred_val)
print('\n class', full_attack_test_class_status)
print('\n mem status', full_attack_test_mem_status)

attack_test_data = (full_attack_test_pred_val, full_attack_test_mem_status,full_attack_test_class_status)

94/94 [==============================] - 0s 1ms/step

 pred (array([[0.858623  , 0.14137688],
       [0.8639384 , 0.13606158],
       [0.6559226 , 0.3440775 ],
       ...,
       [0.81715006, 0.18284997],
       [0.36022046, 0.63977957],
       [0.7021267 , 0.29787335]], dtype=float32), array([[0.9731981 , 0.02680185],
       [0.7135207 , 0.28647932],
       [0.89497554, 0.10502449],
       ...,
       [0.7240938 , 0.27590618],
       [0.73194814, 0.26805174],
       [0.63830024, 0.36169973]], dtype=float32))

 class (array([0, 1, 0, ..., 0, 1, 0]), array([0, 1, 0, ..., 0, 1, 1]))

 mem status (array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([0, 0, 0, ..., 0, 0, 0], dtype=int32))


In [61]:
# save attack test dataset to csv
df = pd.DataFrame()
mem = pd.Series(full_attack_test_pred_val[0][:,0])
nonmem = pd.Series(full_attack_test_pred_val[1][:,0])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
len(total)
df['prob_upper_bound'] = total
df = df.reset_index()
mem = pd.Series(full_attack_test_pred_val[0][:,1])
nonmem = pd.Series(full_attack_test_pred_val[1][:,1])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['prob_lower_bound'] = total

mem = pd.Series(full_attack_test_mem_status[0][:])
nonmem = pd.Series(full_attack_test_mem_status[1][:])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['mem_status'] = total

mem = pd.Series(full_attack_test_class_status[0][:])
nonmem = pd.Series(full_attack_test_class_status[1][:])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['class_status'] = total

df.drop("index", axis=1, inplace=True)


df.to_csv('attack_test_data.csv')

In [43]:
#prepare shadow dataset
n_shadow_models = 10
shadow_data_size = 10000
test_ratio = 0.3

load_shadow_data(shadow_dataset, n_shadow_models, shadow_data_size, test_ratio)

shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000


In [44]:
#train shadow model
per_class_sample=5000
channel=0   
EPS=200
act_layer=3
n_class = 2
VERBOSE = 0
test_ratio = 0.3

n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init = train_shadow_models(n_shadow_models, n_class, dim, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  83.74285697937012 Shadow Test acc :  82.70000219345093
94/94 [==============================] - 0s 1ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  80.94285726547241 Shadow Test acc :  82.23333358764648
94/94 [==============================] - 0s 1ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  80.67142963409424 Shadow Test acc :  80.76666593551636
94/94 [==============================] - 0s 3ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  84.18571352958679 Shadow Test acc :  84.53333377838135
94/94 [==============================] - 0s 2ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  84.6142828464508 Shadow Test acc :  84.33333039283752
94/94 [==============================] - 0s 2ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  76.42857432365417 Shadow Test acc :  76.80000066757202
94/94 [==============================] - 0s 3m

In [46]:
#save the prepared attack data on disk
np.savez(DATA_PATH + 'attack_adult_data.npz', n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init)

/home/sakib/anaconda3/envs/python37/lib/python3.7/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


INFO:tensorflow:Assets written to: ram://9304a9a0-7f19-4390-8231-844c5a5e51cc/assets


In [45]:
# read stored attack model training data 
data_name = 'attack_adult_data.npz'
with np.load(DATA_PATH + data_name, allow_pickle=True) as f:
        n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init = [f['arr_%d' % i] for i in range(len(f.files))]

FileNotFoundError: [Errno 2] No such file or directory: './data/attack_adult_data.npz'

In [66]:
tp, fp, fn, tn, precision, advj, acc, recall, target_membership, predicted_membership, df = train_attack_model(n_attack_data, attack_test_data)
#target_membership, predicted_membership = train_attack_model(n_attack_data, attack_test_data)

Class :  0


AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [54]:
target_membership

array([1, 1, 1, ..., 0, 0, 0], dtype=int32)

In [55]:
predicted_membership

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [133]:
df = pd.DataFrame()
df['target'] = pd.Series(target_membership)
df['predicted'] = pd.Series(predicted_membership)

In [134]:
df

,target,predicted
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
5995,0,1
5996,0,1
5997,0,1
5998,0,1


In [135]:
df.to_csv('test.csv')

In [57]:
attack_data = n_attack_data
check_membership = attack_test_data
n_hidden=50
learning_rate=0.01
batch_size=200
epochs=50
model='nn'
l2_ratio=1e-7

In [58]:
x, y,  classes = attack_data

train_x = x[0]
train_y = y[0]
test_x = x[1]
test_y = y[1]
train_classes = classes[0]
test_classes = classes[1]

#print(tra)


In [73]:
checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership

checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])

train_indices = np.arange(len(train_x))
test_indices = np.arange(len(test_x))
unique_classes = np.unique(train_classes)


predicted_membership, target_membership = [], []

In [74]:
for c in unique_classes:
    print("Class : ", c)
    c_train_indices = train_indices[train_classes == c]
    c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
    c_test_indices = test_indices[test_classes == c]
    c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
    c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)  

    full_cx_data=(c_train_x,c_test_x)
    full_cx_data = np.vstack(full_cx_data)

    full_cy_data=(c_train_y,c_test_y)
    full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])
    full_cy_data = to_categorical(full_cy_data)

    classifier = define_attack_model(2)
    history = classifier.fit(full_cx_data, full_cy_data, epochs=200, batch_size=32, verbose=0)
    #classifier.save('model/attack_model_class{}'.format(c))

    #get predictions on real train and test data
    c_indices = np.where(checkmem_class_status==c)
    predict_y = classifier.predict(checkmem_prediction_vals[c_indices])
    print(predict_y)
    c_pred_y = np.argmax(classifier.predict(checkmem_prediction_vals[c_indices]),axis=1)
    #c_pred_y = np.where(predict_y > 0.5, 1,0)
    #c_pred_y = classifier.predict_classes(checkmem_prediction_vals[c_indices])

    c_target_y = checkmem_membership_status[c_indices]


    target_membership.append(c_target_y)
    predicted_membership.append(c_pred_y)




Class :  0
143/143 [==============================] - 0s 998us/step
[[0.3030294  0.6969707 ]
 [0.304888   0.69511193]
 [0.29782906 0.7021709 ]
 ...
 [0.3023195  0.6976806 ]
 [0.30734053 0.6926593 ]
 [0.30569538 0.69430465]]
143/143 [==============================] - 0s 964us/step
Class :  1
46/46 [==============================] - 0s 1ms/step
[[0.30000547 0.69999444]
 [0.30000547 0.69999444]
 [0.30000547 0.69999444]
 ...
 [0.3000055  0.6999945 ]
 [0.3000055  0.6999945 ]
 [0.3000055  0.6999945 ]]
46/46 [==============================] - 0s 929us/step


In [197]:
c_indices = np.where(checkmem_class_status==c)
predict_y = classifier.predict(checkmem_prediction_vals[c_indices], batch_size=32)
c_pred_y = np.where(predict_y > 0.5, 1,0)
c_target_y = checkmem_membership_status[c_indices]
target_membership.append(c_target_y)
predicted_membership.append(c_pred_y)

48/48 [==============================] - 0s 1ms/step


In [75]:
target_membership=np.array([item for sublist in target_membership for item in sublist])
predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])

In [76]:
prety_print_result (target_membership,predicted_membership)

TP: 2999     FP: 2999     FN: 0     TN: 0
PPV: 0.5000
Advantage: 0.0000


(2999, 2999, 0, 0, 0.5, 0.0, 0.5, 1.0)

In [77]:
np.vstack(predicted_membership).shape

(5998, 1)

In [78]:
df = pd.DataFrame()
df['target'] = pd.Series(target_membership)
df['predicted'] = pd.Series(predicted_membership.reshape((len(predicted_membership))))

In [69]:
df

,target,predicted
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
5993,0,1
5994,0,1
5995,0,1
5996,0,1


In [79]:
predicted_membership

array([1, 1, 1, ..., 1, 1, 1])